In [1]:
print('Link Damping Energy optimization')

Link Damping Energy optimization


In [1]:
import os
import sys
import comtypes.client
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pygad

In [2]:
SapObject = comtypes.client.GetActiveObject("CSI.SAP2000.API.SapObject")
SapModel = SapObject.SapModel
SapModel.SetModelIsLocked(False)

0

In [3]:
def link_option(Y_strength, L_name):
    DOF = [False, False, False, False, False, False]
    Fixed = [False, False, False, False, False, False]
    NonLinear = [False, False, False, False, False, False]
    Ke = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Ce = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    K =[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Yield = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Ratio = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    Exp =[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

    DOF[0] = True
    Fixed[0] = True

    DOF[1] = True
    Fixed[1] = True

    DOF[2] = True
    Fixed[2] = True

    DOF[3] = True
    Fixed[3] = True

    DOF[3] = True
    Fixed[3] = True

    DOF[4] = True
    Fixed[4] = True

    DOF[5] = True
    NonLinear[5] = True
    Ke[5] = 50000
    Ce[5] = 0
    K[5] = 50000
    Yield[5] = Y_strength
    Ratio[5] = 0.03
    Exp[5] = 2
    ret = SapModel.PropLink.SetPlasticWen(L_name, DOF, Fixed, NonLinear, Ke, Ce, K, Yield, Ratio, Exp, 2, 0)


def get_data(L_num, save_path,Plot_graph = True):
    eItemTypeElm = 1
    NumberResults = 0
    Obj =[]
    Elm =[]
    PointElm =[]
    LoadCase =[]
    StepType =[]
    StepNum =[]
    P =[]
    V2 =[]
    V3 =[]
    T =[]
    M2 =[]
    M3 =[]

    U1=[]
    U2=[]
    U3=[]
    R1=[]
    R2=[]
    R3=[]
    SapModel.Results.Setup.SetOptionDirectHist(2)
    [NumberResults, Obj, Elm, LoadCase,PointElm, StepType, StepNum, P, V2, V3, T, M2, M3, ret] = SapModel.Results.LinkForce(L_num, eItemTypeElm, NumberResults, Obj, Elm, PointElm, LoadCase, StepType, StepNum, P, V2, V3, T, M2, M3)

    [NumberResults, Obj, Elm, LoadCase, StepType, StepNum, U1, U2, U3, R1, R2, R3, ret]= SapModel.Results.LinkDeformation(L_num, eItemTypeElm, NumberResults, Obj, Elm, LoadCase, StepType, StepNum, U1, U2, U3, R1, R2, R3)

    result = pd.DataFrame()

    result['Desplacment'] = R3
    result['Force'] = M3[0:len(M3):2]

    if Plot_graph:
        plt.plot(result['Desplacment'], result['Force'])
        plt.grid()
        save_path= save_path + '/'+ str(L_num) + ".png"
        plt.savefig(save_path)
        plt.close()


    #Energy = np.abs(result['Desplacment'][len(result)-1]*result['Force'][0] - result['Desplacment'][0]*result['Force'][len(result)-1])/2
    Energy = 0
    for i in range(len(result)-1):
        #Area = result['Desplacment'][i]*result['Force'][i+1] - result['Desplacment'][i+1]*result['Force'][i]
        Area = (result['Force'][i] + result['Force'][i+1]) / 2 * (result['Desplacment'][i+1]- result['Desplacment'][i])
        Energy = Energy +  np.abs(Area)
    return result, Energy

In [4]:
def main(Y_strength):
    print(Y_strength)
    SapModel.SetModelIsLocked(False)
    Link_list = ['N-LIN1', 'LIN2','LIN3','LIN4','LIN5']
    Link_numbers = ['1', '3', '5', '7', '9', '11', '13', '15', '17', '19']
    #Link_numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

    for link_name in Link_list:
        link_option( Y_strength[Link_list.index(link_name)], link_name)
    
    SapModel.Analyze.RunAnalysis()
    SapModel.Results.Setup.DeselectAllCasesAndCombosForOutput()
    ret = SapModel.Results.Setup.SetCaseSelectedForOutput("ACASE1")

    E = []
    plot_dirc = os.path.join("plots/", str(Y_strength)) 
    os.makedirs(plot_dirc, exist_ok=True) 
    for num in Link_numbers:
        r, e = get_data(num, plot_dirc)
        E.append(e)
    print("E=",np.sum(E)/3)

    return np.sum(E)
#23, 95

In [40]:
Y_strength = [100, 100, 100, 100, 100]

print("Initial Energy value : ",main(Y_strength))

[100, 100, 100, 100, 100]
E= 18.518584919129605
Initial Energy value :  55.55575475738881


In [15]:
def fitness_func(solution, solution_idx):
    return 1.0/main(solution)

last_fitness = 0
def on_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness))
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

fitness_function = fitness_func

num_generations = 8 #iteratinos
num_parents_mating = 3

sol_per_pop = 5 # solutions per iteration
num_genes = len(Y_strength)

init_range_low = 10
init_range_high = 300

parent_selection_type = "sss"
keep_parents = 1

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 20

ga_instance = pygad.GA(#on_generation=on_generation,
                        num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes
                       )

In [16]:
ga_instance.run()

[126.00458162  61.87545117 138.69272586 146.32232379 277.96807629]
E= 17.68201958290213
[ 19.98515311 143.629274    65.46124574 204.15189615 244.66295272]
E= 14.319660683159606
[172.18382288 202.42265894 219.35659102  24.58706803 168.34994033]
E= 18.62937302166282
[270.01256981  80.22777326 178.64694782  13.39998097  67.29279118]
E= 16.726370268010538
[258.82043637  21.64986307 147.35158    266.38558113 147.62331769]
E= 14.634720183517736
[ 19.32331548 143.629274    65.46124574 266.38558113 147.62331769]
E= 14.23495561813712
[258.82043637  21.64986307 147.35158    266.38558113  68.00815061]
E= 14.646668429878012
[270.01256981 143.49249197  65.46124574 204.15189615 244.66295272]
E= 19.152158715892593
[ 19.98515311  21.64986307 147.2571296  266.38558113 147.62331769]
E= 11.258981099819186
[ 19.98515311  21.64986307  65.46124574 266.38558113 146.85244824]
E= 11.12412638608924
[ 19.11291205 143.629274    65.46124574 204.15189615 244.66295272]
E= 14.214049656013083
[ 19.14728673 143.629274 

In [ ]:
"""
solutions 
[100, 100, 100, 100, 100]
E= 19.33

[23.99676231, 65.60974758, 259.04204045, 262.65059394, 38.30031988]
E= 21.73

[15.45859686, 141.18188102, 129.69220784, 32.57203134, 105.8820149]
E= 20.41

[ 20.42899259  20.56412642  65.46124574 265.80892073 147.47263335]
E= 26.65
"""